In [2]:
import tensorflow as tf
import numpy as np
import os

work_directory = os.path.abspath(os.path.dirname(__name__))
data_directory = os.path.join(work_directory, 'data')

# Chargement des données

In [ ]:
cifar_map = {
    {'train': {
        'images': None,
        'labels': None
    }},
    {'test': {
        'images': None,
        'labels': None
    }}
}

with open('data/', 'r') as f:
    pass